# iris center - UNet

# import

In [1]:
import keras
from keras import backend as K

def load_modules():
    import sys
    import numpy as np
    import warnings
    
    sys.path.append("../../../")
    warnings.filterwarnings("ignore")
    np.set_printoptions(precision=6, suppress=True)
    
    %run ../../../ds/icontour/npz/gen.py
    %run ../../../ac/visualizer/plotter.py
    %run ../../../ai/model/iris/unet/iris.py
    %run ../../../ai/libs/keras/callbacks/history.py
    %run ../../../ai/libs/keras/callbacks/stopper.py
    %run ../../../ai/libs/keras/callbacks/tensorboard.py
    %run ../../../ai/libs/keras/callbacks/checkpoint.py
        
load_modules()

Using TensorFlow backend.


# backbone model

In [2]:
INPUT_SHAPE=(112, 112, 3)
im = make_uiris_net(input_shape=INPUT_SHAPE)
im.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 112, 112, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 32) 9248        dropout_1[0][0]                  
__________________________________________________________________________________________________
max_poolin

# callback

In [3]:
chk_path = "/home/chy/archive-model/incubator/iris-contour/iris-u-{epoch:02d}-{val_loss:.7f}.hdf5"

history = TimeHistory()
checkpoint = model_checkpoint(chk_path)
tensorboard = TensorBoard()
stopper = EarlyStopping(monitor='val_loss', 
                        min_delta=0, 
                        patience=5, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True)

callbacks = [history, checkpoint, stopper, tensorboard]

In [4]:
EXP_CODE = "pilot-uiris"
NUM_EPOCH = 20
BATCH_SIZE = 64

# model save

In [ ]:
model_json = im.to_json()
with open("./iris-{}.json".format(EXP_CODE), "w") as json_file : 
    json_file.write(model_json)

# train report

#### Table
|exp|aug|rot|type|comment|
|---|---|---|---|---|
|pilot|x|x|타원 파라미터 추론||

# train

In [ ]:
# npz_path = "/media/chy/1326657F605F16F2/processed/iris-contour-npz/"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity-partial"
npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc-partial"

gen_train = IrisContourGenerator(npz_base_path=npz_path, 
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.TRAIN,
                                 is_ellipse_mode=False,
                                 use_aug=True)

gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.VALID,
                                 is_ellipse_mode=False,
                                 use_aug=False)

*** meta verify complete [Unknown] ***
*** meta verify complete [Unknown] ***


In [ ]:
im.fit_generator(generator=gen_train,
                 validation_data=gen_valid,
                 callbacks=callbacks,
                 epochs=NUM_EPOCH,
                 workers=16, 
                 use_multiprocessing=True,
                 shuffle=True)

Epoch 1/20
31242/31242 [==============================] - 43683s 1s/step - loss: 0.5346 - acc: 0.8805 - val_loss: 0.5066 - val_acc: 0.8981
Epoch 2/20
 3076/31242 [=>............................] - ETA: 9:02:36 - loss: 0.5098 - acc: 0.8961

# test

In [ ]:
gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                  batch_size=BATCH_SIZE, 
                                  purpose=Purpose.TEST,
                                  is_ellipse_mode=False,
                                  use_aug=True)